In [1]:
import pandas as pd
from pathlib import Path
from updates_join_annotations import open_df_from_tsv
import updates_get_average_scores_per_label as scores


In [2]:
df = pd.read_pickle('token_level_df.pkl')
df.info()
annotator_names = ['avelli', 'bos', 'meskers']
#annotator_names = ['avelli', 'bos', 'katsburg', 'meskers', 'opsomer', 'swartjes', 'vanderpas', 'vervaart']
print(df.shape)
df

<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 2503_0 to 2570_54
Data columns (total 14 columns):
token_d_id          127 non-null float64
token               127 non-null object
labels_avelli       127 non-null object
relation_avelli     127 non-null object
temp                127 non-null object
file_id             127 non-null object
sent_id             127 non-null object
token_s_id          127 non-null object
labels_bos          127 non-null object
relation_bos        127 non-null object
temp                127 non-null object
labels_meskers      127 non-null object
relation_meskers    127 non-null object
temp                127 non-null object
dtypes: float64(1), object(13)
memory usage: 14.9+ KB
(127, 14)


,token_d_id,token,labels_avelli,relation_avelli,temp,file_id,sent_id,token_s_id,labels_bos,relation_bos,temp,labels_meskers,relation_meskers,temp
2503_0,0.0,NF,_,_,_,2503,1,1,_,_,_,_,_,_
2503_1,1.0,:,_,_,_,2503,1,2,_,_,_,_,_,_
2503_2,2.0,Pijn,_,_,_,2503,1,3,_,_,_,_,_,_
2503_3,3.0,in,_,_,_,2503,1,4,_,_,_,_,_,_
2503_4,4.0,de,_,_,_,2503,1,5,_,_,_,_,_,_
2503_5,5.0,rug,_,_,_,2503,1,6,_,_,_,_,_,_
2503_6,6.0,",",_,_,_,2503,1,7,_,_,_,_,_,_
2503_7,7.0,verlicht,_,_,_,2503,1,8,_,_,_,_,_,_
2503_8,8.0,met,_,_,_,2503,1,9,_,_,_,_,_,_
2503_9,9.0,warme,_,_,_,2503,1,10,_,_,_,_,_,_


In [3]:
def change_labels_to_list_format(df):
    for column_name in df.columns:
        if column_name.startswith('labels_'):
            # to avoid the function repeating itself:
            if type(df[column_name][0]) == list:
                continue
            original_labels = df[column_name]

            cleaned_labels = []
            for label in original_labels:
                if '|' in label:
                    label_split = label.split('|')
                else:
                    label_split = [label]
                cleaned_labels.append(label_split)

            df[column_name] = cleaned_labels
    return df

In [ ]:
# Not using this code at the moment
class Span:
    """
    class containing information about the Span
    """

    def __init__(self,
                 begin, end, token):
        self.begintoken = begin
        self.endtoken = end
        self.token = token

In [4]:
def get_dict_per_file_per_annotator(df, annotator, filename):
    df_filtered = df.loc[df['file_id'] == filename][['token_d_id', 'token', f'labels_{annotator}']]


    labels_per_doc_per_annotator_dict = dict()

    for index, row in df_filtered.iterrows():
        if row[f'labels_{annotator}'] != ['_']:

            for label_and_id in row[f'labels_{annotator}']:
                if label_and_id in labels_per_doc_per_annotator_dict:
                    labels_per_doc_per_annotator_dict[label_and_id]['indices'].append(row['token_d_id'])
                    labels_per_doc_per_annotator_dict[label_and_id]['tokens'].append(row['token'])
                else:
                    labels_per_doc_per_annotator_dict[label_and_id] = {'indices': [row['token_d_id']],
                                                                      'tokens': [row['token']],
                                                                      'file_id': filename,
                                                                      'annotator': annotator}
                    
    return labels_per_doc_per_annotator_dict

In [31]:
def join_dicts_on_label_and_index(dict_of_dicts):

    gathered_dict = dict()

    for uncleaned_dict in dict_of_dicts.values():
        for key, inner_dict in uncleaned_dict.items():

            start = inner_dict['indices'][0]
            if inner_dict['indices'] == 1:
                end = inner_dict['indices'][0]
            else:
                end = inner_dict['indices'][-1]


            cleaned_inner_dict = {'file_id': inner_dict['file_id'],
                                    'start_index': start, 
                                  'end_index': end,
                                 'tokens': inner_dict['tokens'],
                                 'annotators': set([inner_dict['annotator']])}

            cleaned_key = '_'.join([inner_dict['file_id'], str(start), str(end), key.split('[')[0]])

            if cleaned_key in gathered_dict:
                gathered_dict[cleaned_key]['annotators'].add(inner_dict['annotator'])
            else:
                gathered_dict[cleaned_key] = cleaned_inner_dict

    return gathered_dict

In [32]:
# Change format of labels in f'labels_{name}' columns to list format
df = change_labels_to_list_format(df)

In [33]:
# Create dictionary of dictionary separated by annotators and files

dict_of_dicts_per_file_annotator = dict()
for annotator in annotator_names:
    for file_id in set(df['file_id']):
        dict_of_dicts[f'{annotator}_{file_id}'] = get_dict_per_file_per_annotator(df, annotator, file_id)
        
for key in dict_of_dicts:
    print(key)

avelli_2503
avelli_2570
avelli_2546
bos_2503
bos_2570
bos_2546
meskers_2503
meskers_2570
meskers_2546


In [35]:
# Join the dictionaries to form one cleaned dictionary with identifiers containing file_id and start, end index
data = join_dicts_on_label_and_index(dict_of_dicts)

In [36]:

df_new = pd.DataFrame.from_dict(data, orient='index')
df_new
df_new['annotator_count'] = df_new.apply(lambda row: len(row['annotators']), axis=1)
df_new

,file_id,start_index,end_index,tokens,annotators,annotator_count
2503_16.0_16.0_.B152: Stemming,2503,16.0,16.0,[emotioneel],"{bos, avelli}",2
2503_16.0_16.0_STM 1,2503,16.0,16.0,[emotioneel],"{bos, avelli}",2
2503_18.0_26.0_stm\_reaction,2503,18.0,26.0,"[schoonzoon, hoort, morgen, de, uitslag, van, ...",{avelli},1
2503_29.0_33.0_.B152: Stemming,2503,29.0,33.0,"[maakt, zich, hier, veel, zorgen]",{avelli},1
2503_29.0_34.0_.B152: Stemming,2503,29.0,34.0,"[maakt, zich, hier, veel, zorgen, om]",{bos},1
2503_32.0_32.0_STM 0,2503,32.0,32.0,[veel],{bos},1
2503_32.0_33.0_STM 0,2503,32.0,33.0,"[veel, zorgen]",{avelli},1
2546_0.0_12.0_type\_Background,2546,0.0,12.0,"[Op, Dd-mm-jjjj, zag, ik, bovengenoemde, patiÃ...",{avelli},1
2546_14.0_35.0_type\_Background,2546,14.0,35.0,"[80, mg, prednison, als, gevolg, van, ipi/nivo...","{bos, avelli, meskers}",3
2546_26.0_26.0_.D450: Lopen en zich verplaatsen,2546,26.0,26.0,[lopen],{meskers},1
